<a href="https://colab.research.google.com/github/ApurbaPaul-NLP/BERT_and_HUGGINGFACE/blob/main/Prog2_HuggingFace_NER_Token_Classification_08_09_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install transformers
! pip install datasets
! pip install seqeval
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 30.0 MB/s 
     |████████████████████████████████| 120 kB 74.2 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 19.5 MB/s 
     |████████████████████████████████| 115 kB 67.5 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 127 kB 70.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sh

In [6]:
import transformers

print(transformers.__version__)

4.21.3


# **Fine-tuning a model on a token classification task**



```
The most common token classification tasks are:

    NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).


    POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)

    Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for 
these kinds of tasks and use Keras to fine-tune a model on it.
```



In [7]:
task = "ner"  # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

**Loading the dataset**

We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). 

This can be easily done with the functions load_dataset and load_metric.

In [8]:
from datasets import load_dataset, load_metric
datasets = load_dataset("conll2003")
datasets

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

To access an actual element, you need to select a split first, then give an index:

In [9]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the features of the dataset:

In [10]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)



```
So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:

'PER' for person
'ORG' for organization
'LOC' for location
'MISC' for miscellaneous
```



Since the labels are lists of ClassLabel, the actual names of the labels are nested in the feature attribute of the object above:

In [11]:
tasks=["ner",'pos','chunk']
for task in tasks:
  label_list = datasets["train"].features[f"{task}_tags"].feature.names
  print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP']


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [12]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])    

,id,tokens,pos_tags,chunk_tags,ner_tags
0,3953,"[Strasbourg, 1, (, Zitelli, 80th, ), Le, Havre, 0, .]","[NNP, CD, (, NNP, CD, ), NNP, NNP, CD, .]","[B-NP, I-NP, O, B-NP, I-NP, O, B-NP, I-NP, I-NP, O]","[B-ORG, O, O, B-PER, O, O, B-ORG, I-ORG, O, O]"
1,13101,"[They, contained, $, 650,000, in, jewelry, and, $, 40,000, in, cash, ,, Andrews, said, .]","[PRP, VBD, $, CD, IN, NN, CC, $, CD, IN, NN, ,, NNP, VBD, .]","[B-NP, B-VP, B-NP, I-NP, B-PP, B-NP, O, B-NP, I-NP, B-PP, B-NP, O, B-NP, B-VP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, B-PER, O, O]"
2,6758,"[for, ,, against, ,, points, )]","[IN, ,, RB, ,, VBZ, )]","[B-PP, O, B-ADVP, O, B-VP, O]","[O, O, O, O, O, O]"
3,8744,"[Czech, Republic, 1, 0, 0, 1, 3, 7, 0]","[NNP, NNP, CD, CD, CD, CD, CD, CD, CD]","[B-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP, I-NP]","[B-LOC, I-LOC, O, O, O, O, O, O, O]"
4,2240,"[The, Russian, ate, up, the, ground, in, a, swift, last, lap, to, shave, 0.36, seconds, off, the, previous, best, of, 2:29.34, set, by, Mozambique, 's, Maria, Mutola, in, the, same, stadium, in, August, last, year, .]","[DT, NNPS, VBD, RP, DT, NN, IN, DT, NN, JJ, NN, TO, VB, CD, NNS, IN, DT, JJ, JJS, IN, CD, NN, IN, NNP, POS, NNP, NNP, IN, DT, JJ, NN, IN, NNP, JJ, NN, .]","[B-NP, I-NP, B-VP, B-PRT, B-NP, I-NP, B-PP, B-NP, I-NP, B-NP, I-NP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, B-VP, B-PP, B-NP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, B-NP, I-NP, O]","[O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O]"
5,12221,"[Goosen, (, South, Africa, ), 71, 74, ,, Ove, Sellberg, (, Sweden, ), 71, 74, ,]","[NNP, (, NNP, NNP, ), CD, CD, ,, NNP, NNP, (, NNP, ), CD, CD, ,]","[B-NP, O, B-NP, I-NP, O, B-NP, B-NP, O, B-NP, I-NP, O, B-NP, O, B-NP, B-NP, O]","[B-PER, O, B-LOC, I-LOC, O, O, O, O, B-PER, I-PER, O, B-LOC, O, O, O, O]"
6,1150,"[Ivanisevic, hit, 32, aces, but, was, outplayed, from, the, back, court, by, the, 24-year-old, Tillstrom, .]","[NNP, VBD, CD, NNS, CC, VBD, VBN, IN, DT, JJ, NN, IN, DT, JJ, NNP, .]","[B-NP, B-VP, B-NP, I-NP, O, B-VP, I-VP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, O]"
7,9462,"["", We, joined, the, prayer, to, express, our, solidarity, with, her, work, for, the, cause, of, the, poor, and, downtrodden, ,, "", said, Nanda, Gopal, Bhattacharya, ,, a, communist, minister, in, West, Bengal, .]","["", PRP, VBD, DT, NN, TO, VB, PRP$, NN, IN, PRP$, NN, IN, DT, NN, IN, DT, JJ, CC, JJ, ,, "", VBD, NNP, NNP, NNP, ,, DT, JJ, NN, IN, NNP, NNP, .]","[O, B-NP, B-VP, B-NP, I-NP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O, B-VP, B-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, I-PER, O, O, O, O, O, B-LOC, I-LOC, O]"
8,4455,"[Internet, Startup, funded, to, develop, Java, software, .]","[NNP, NNP, VBN, TO, VB, NNP, NN, .]","[B-NP, I-NP, B-VP, I-VP, I-VP, B-NP, I-NP, O]","[B-ORG, I-ORG, O, O, O, B-MISC, O, O]"
9,10788,"[PARIS, 1996-08-28]","[NNP, CD]","[B-NP, I-NP]","[B-LOC, O]"


In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [14]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [15]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 1037, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument is_split_into_words=True:

In [16]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [17]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [18]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a [CLS] and a [SEP] above) and then because of those possible splits of words in multiple tokens:

In [19]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [20]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [21]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [22]:
aligned_labels

[-100,
 11,
 11,
 11,
 12,
 13,
 11,
 12,
 12,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 21,
 13,
 11,
 12,
 21,
 22,
 11,
 11,
 11,
 13,
 11,
 1,
 13,
 11,
 17,
 11,
 12,
 12,
 21,
 1,
 0,
 -100]

Here we set the labels of all special tokens to -100 (the index that is ignored by the loss function) and the labels of all other tokens to the label of the word they come from. 

Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. 

Both strategies are possible with this script; simply change the value of this flag to swap between them

In [23]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [24]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [25]:
tokenize_and_align_labels(datasets["train"][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 11, 21, 11, 12, 21, 22, 11, 12, 0, -100], [-100, 11, 12, -100], [-1

In [26]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [27]:
tokenized_datasets["train"]["labels"][0]

[-100, 11, 21, 11, 12, 21, 22, 11, 12, 0, -100]

**Fine-tuning the model**

Now that our data is ready, we can download the pretrained model and fine-tune it. 

Since all our tasks are about token classification, we use the TFAutoModelForTokenClassification class. 

Like with the tokenizer, the from_pretrained method will download and cache the model for us. 

The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before). 

We can also set the id2label and label2id properties for our model - this is optional, but will give us some nice cleaner outputs later.

In [28]:
from transformers import TFAutoModelForTokenClassification

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

After all of the Tensorflow initialization messages, we see a warning that is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). 

This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To compile a Keras Model, we will need to set an optimizer and our loss function. 

We can use the create_optimizer function from Transformers. 

Here we tweak the learning rate, use the batch_size defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. 

This function will also create and apply a learning rate decay schedule for us.

In [29]:
from transformers import create_optimizer

num_train_epochs = 3
num_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [30]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="tf")

In [32]:
train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

validation_set = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [33]:
import numpy as np
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=validation_set
)

In [34]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_checkpoint.split("/")[-1]
# push_to_hub_model_id = f"{model_name}-finetuned-{task}"

# tensorboard_callback = TensorBoard(log_dir="./tc_model_save/logs")

# push_to_hub_callback = PushToHubCallback(
#     output_dir="./tc_model_save",
#     tokenizer=tokenizer,
#     hub_model_id=push_to_hub_model_id,
# )

callbacks = [metric_callback]   #, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

Epoch 1/3
877/877 [==============================] - ETA: 0s - loss: 0.4160

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


877/877 [==============================] - 125s 130ms/step - loss: 0.4160 - val_loss: 0.2289 - precision: 0.8971 - recall: 0.8934 - f1: 0.8952 - accuracy: 0.9421
Epoch 2/3
877/877 [==============================] - 111s 126ms/step - loss: 0.1996 - val_loss: 0.1969 - precision: 0.9066 - recall: 0.9048 - f1: 0.9057 - accuracy: 0.9485
Epoch 3/3
877/877 [==============================] - 110s 126ms/step - loss: 0.1644 - val_loss: 0.1937 - precision: 0.9106 - recall: 0.9058 - f1: 0.9082 - accuracy: 0.9495


In [35]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

# You can, of course, use your own username and model name here 
# once you've pushed your model using the code above!
checkpoint = "Rocketknight1/distilbert-base-uncased-finetuned-ner"
model = TFAutoModelForTokenClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Some layers from the model checkpoint at Rocketknight1/distilbert-base-uncased-finetuned-ner were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at Rocketknight1/distilbert-base-uncased-finetuned-ner and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
sample_sentence = "Britain will send scores of artillery guns and more than 1,600 anti-tank weapons to Ukraine in the latest supply of western arms to help bolster its defence against Russia, the UK defence secretary, Ben Wallace, said on Thursday."
tokenized = tokenizer([sample_sentence], return_tensors="np")

In [37]:
import numpy as np
outputs = model(tokenized).logits
classes = np.argmax(outputs, axis=-1)[0]
print(classes)

[0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 7 0 0 0 0 0 0 0 0 5 0 0
 5 0 0 0 1 2 0 0 0 0 0 0]


In [38]:
outputs = [(tokenizer.decode(token), model.config.id2label[id]) for token, id in zip(tokenized["input_ids"][0], classes)]
print(outputs)

[('[CLS]', 'O'), ('britain', 'B-LOC'), ('will', 'O'), ('send', 'O'), ('scores', 'O'), ('of', 'O'), ('artillery', 'O'), ('guns', 'O'), ('and', 'O'), ('more', 'O'), ('than', 'O'), ('1', 'O'), (',', 'O'), ('600', 'O'), ('anti', 'O'), ('-', 'O'), ('tank', 'O'), ('weapons', 'O'), ('to', 'O'), ('ukraine', 'B-LOC'), ('in', 'O'), ('the', 'O'), ('latest', 'O'), ('supply', 'O'), ('of', 'O'), ('western', 'B-MISC'), ('arms', 'O'), ('to', 'O'), ('help', 'O'), ('bo', 'O'), ('##lster', 'O'), ('its', 'O'), ('defence', 'O'), ('against', 'O'), ('russia', 'B-LOC'), (',', 'O'), ('the', 'O'), ('uk', 'B-LOC'), ('defence', 'O'), ('secretary', 'O'), (',', 'O'), ('ben', 'B-PER'), ('wallace', 'I-PER'), (',', 'O'), ('said', 'O'), ('on', 'O'), ('thursday', 'O'), ('.', 'O'), ('[SEP]', 'O')]


In [39]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification",
    "Rocketknight1/distilbert-base-uncased-finetuned-ner",
    framework="tf"
)

Some layers from the model checkpoint at Rocketknight1/distilbert-base-uncased-finetuned-ner were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at Rocketknight1/distilbert-base-uncased-finetuned-ner and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
token_classifier(sample_sentence)

[{'entity': 'B-LOC',
  'score': 0.9885372,
  'index': 1,
  'word': 'britain',
  'start': 0,
  'end': 7},
 {'entity': 'B-LOC',
  'score': 0.99329174,
  'index': 19,
  'word': 'ukraine',
  'start': 84,
  'end': 91},
 {'entity': 'B-MISC',
  'score': 0.6854809,
  'index': 25,
  'word': 'western',
  'start': 116,
  'end': 123},
 {'entity': 'B-LOC',
  'score': 0.9881704,
  'index': 34,
  'word': 'russia',
  'start': 165,
  'end': 171},
 {'entity': 'B-LOC',
  'score': 0.54583544,
  'index': 37,
  'word': 'uk',
  'start': 177,
  'end': 179},
 {'entity': 'B-PER',
  'score': 0.98485094,
  'index': 41,
  'word': 'ben',
  'start': 199,
  'end': 202},
 {'entity': 'I-PER',
  'score': 0.9929636,
  'index': 42,
  'word': 'wallace',
  'start': 203,
  'end': 210}]